# Federated Learning - SMS spam prediction with a GRU model

In this notebook, we will train a model using federated approach.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:8080)
 - Grid Node Bob (http://localhost:3000)
 - Grid Node Anne (http://localhost:3001)
 
To **start the gateway**:
* ```cd gateway```
* ```python gateway.py --start_local_db --port=8080```

To **start one grid node**:

* ```cd app/websocket/```

* ```python websocket_app.py --start_local_db --id=anne --port=3001 --gateway_url=http://localhost:8080```
 
This notebook was made based on [Federated SMS Spam prediction](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials/advanced/Federated%20SMS%20Spam%20prediction).

Authors:
* André Macedo Farias: Github: [@andrelmfarias](https://github.com/andrelmfarias) | Twitter: [@andrelmfarias](https://twitter.com/andrelmfarias)
* George Muraru: Githib [@gmuraru](https://github.com/gmuraru) | Twitter: [@georgemuraru](https://twitter.com/georgemuraru) | Facebook: [@George Cristian Muraru](https://www.facebook.com/georgecmuraru)

## Useful imports

In [408]:
import numpy as np

import syft as sy
import grid as gr

import torch as th
from torch import optim

import warnings

warnings.filterwarnings("ignore")

## Connecting to Grid Network

In [409]:
hook = sy.TorchHook(th)

my_grid = gr.GridNetwork("http://localhost:8080")

## Seach a dataset

In [410]:
data = my_grid.search("#X", "#spam", "#dataset")
target = my_grid.search("#Y", "#spam", "#dataset")

In [411]:
data

[[(Wrapper)>[PointerTensor | me:53716591731 -> bob:81837175413]],
 [(Wrapper)>[PointerTensor | me:70032216725 -> anne:44834606227]]]

In [412]:
target

[[(Wrapper)>[PointerTensor | me:70943240410 -> bob:41763767662]],
 [(Wrapper)>[PointerTensor | me:20617297975 -> anne:46155488760]]]

## Load the model

In [413]:
from handcrafted_GRU import GRU

### Parameters

In [414]:
VOCAB_SIZE = 0
for data_comp in data:
    VOCAB_SIZE = max(VOCAB_SIZE, int(data_comp[0].max().get()))
    
VOCAB_SIZE += 1
HIDDEN_DIM = 10
EMBEDDING_DIM = 50
BATCH_SIZE = 128
CLIP = 5 # gradient clipping - to avoid gradient explosion (frequent in RNNs)
DROPOUT = 0.2
EPOCHS = 15
LR = 0.1

In [415]:
# Initiating the model
model = GRU(vocab_size=VOCAB_SIZE, hidden_dim=HIDDEN_DIM, embedding_dim=EMBEDDING_DIM, dropout=DROPOUT)

# And the optimizer
optimizer = optim.SGD(model.parameters(), lr=LR)

# And the loss
criterion = nn.BCELoss()

## Perform train

In [416]:
import math # Needed for separating into batches

def train(epoch):
    dataset_size = sum([len(data[i][0]) for i in range(len(data))])
    model.train()
    
    for i in range(len(data)):
        loss_cum = 0
        nr_batches = math.ceil(len(data[i][0]) / BATCH_SIZE)
        for batch_idx in range(nr_batches):
            # Extract the batch for training and target
            data_batch = data[i][0][BATCH_SIZE * batch_idx : BATCH_SIZE * (batch_idx + 1), :]
            target_batch = target[i][0][BATCH_SIZE * batch_idx : BATCH_SIZE * (batch_idx + 1)]
            
            # Send the model to the worker
            worker = data_batch.location
            model.send(worker)
            h = th.Tensor(np.zeros((data_batch.shape[0], HIDDEN_DIM))).send(worker)
            
            optimizer.zero_grad()
            pred, _ = model(data_batch, h)
            loss = criterion(pred.squeeze(), target_batch.float())
            loss.backward()
            optimizer.step()
            model.get()
            
            # Cumulate the loss
            loss_cum += loss.get().item()
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * data[i][0].shape[0], dataset_size,
                       100. *  (i * data[i][0].shape[0]) / dataset_size, loss_cum))

for epoch in range(EPOCHS):
    train(epoch)

Train Epoch: 0 [0/5572 (0%)]	Loss: 14.413047
Train Epoch: 0 [2786/5572 (50%)]	Loss: 10.590258
Train Epoch: 1 [0/5572 (0%)]	Loss: 9.518043
Train Epoch: 1 [2786/5572 (50%)]	Loss: 9.040479
Train Epoch: 2 [0/5572 (0%)]	Loss: 8.841945
Train Epoch: 2 [2786/5572 (50%)]	Loss: 8.660154
Train Epoch: 3 [0/5572 (0%)]	Loss: 8.662314
Train Epoch: 3 [2786/5572 (50%)]	Loss: 8.526029
Train Epoch: 4 [0/5572 (0%)]	Loss: 8.523282
Train Epoch: 4 [2786/5572 (50%)]	Loss: 8.358135
Train Epoch: 5 [0/5572 (0%)]	Loss: 8.402678
Train Epoch: 5 [2786/5572 (50%)]	Loss: 8.199387
Train Epoch: 6 [0/5572 (0%)]	Loss: 8.255481
Train Epoch: 6 [2786/5572 (50%)]	Loss: 8.066721
Train Epoch: 7 [0/5572 (0%)]	Loss: 8.121440
Train Epoch: 7 [2786/5572 (50%)]	Loss: 7.857515
Train Epoch: 8 [0/5572 (0%)]	Loss: 7.965773
Train Epoch: 8 [2786/5572 (50%)]	Loss: 7.629926
Train Epoch: 9 [0/5572 (0%)]	Loss: 7.752881
Train Epoch: 9 [2786/5572 (50%)]	Loss: 7.477828
Train Epoch: 10 [0/5572 (0%)]	Loss: 7.517532
Train Epoch: 10 [2786/5572 (50%)]